In [13]:
import numpy as np
if __name__ == "__main__":
    #   Each module has dimension 20x20x8cm and weighs 135g
    x,y,z = 0.2,0.2,0.08
    m = 0.135

    option=2        # crazyflie1, crazyflie2, 2boxes
    
    if option == 0:
        Ixx, Iyy, Izz = 1.4e-5, 1.4e-5, 2.17e-5
    elif option == 1:
        Ixx, Iyy, Izz = 2.3951e-5, 2.3951e-5, 3.2347e-5
    else:
        Ixx = 1/12*m*(z**2+y**2)
        Iyy = 1/12*m*(z**2+x**2)
        Izz = 1/12*m*(x**2+y**2)

    I = np.eye(3)
    I[0,:] *= Ixx
    I[1,:] *= Iyy
    I[2,:] *= Izz
    diag = np.zeros((3,3))
    
    # '''
    # 2x1

    # 1 - 0
    # '''
    # xis = np.array([0,0])
    # yis = np.array([0.1, -0.1])     
    # diag[0,0] = np.sum(yis**2)
    # diag[1,1] = np.sum(xis**2)
    # diag[2,2] = np.sum(xis**2+yis**2)
    # Is = len(xis)*I
    # Is += m*diag
    # print(Is[0,0], Is[1,1], Is[2,2])

    # '''
    # 2x2
    # 3 - 2
    # |   |
    # 1 - 0
    # '''
    # xis = np.array([0.1,-0.1])
    # yis = np.array([0.1,-0.1])     
    # diag[0,0] = np.sum(yis**2)
    # diag[1,1] = np.sum(xis**2)
    # diag[2,2] = np.sum(xis**2+yis**2)
    # Is = len(xis)*I
    # Is += m*diag
    # mass = m*4

    # '''
    # 3x3
    # 8 - 7 - 6
    # |   |   |
    # 5 - 4 - 3
    # |   |   |
    # 2 - 1 - 0
    # '''
    # xis = np.array([0.2,0.0,-0.2])      # mod this
    # yis = np.array([0.2,0.0,-0.2]*3)      # mod this
    # xis = np.repeat(xis, 3)           # mod this
    # diag[0,0] = np.sum(yis**2)
    # diag[1,1] = np.sum(xis**2)
    # diag[2,2] = np.sum(xis**2+yis**2)
    # Is = len(xis)*I
    # Is += m*diag
    # mass = m*9                          # mod this



    # '''
    # plus
    #     3
    #     |
    # 4 - 0 - 2
    #     |
    #     1
    # '''
    # xis = np.array([0,0.2,0,-0.2,0])
    # yis = np.array([0,0,0.2,0,-0.2])     
    # diag[0,0] = np.sum(yis**2)
    # diag[1,1] = np.sum(xis**2)
    # diag[2,2] = np.sum(xis**2+yis**2)
    # Is = len(xis)*I
    # Is += m*diag
    # mass = m*5


    '''
    TETRIS
    3 - 0 - 2
        |
        1
    '''
    xis = np.array([0,0.2,0,0])
    yis = np.array([0,0,0.2,-0.2])     
    diag[0,0] = np.sum(yis**2)
    diag[1,1] = np.sum(xis**2)
    diag[2,2] = np.sum(xis**2+yis**2)
    Is = len(xis)*I
    Is += m*diag
    mass = m*4

    
    # '''
    # 4x4
    # 15 - 14 - 13 - 12
    # |    |    |    | 
    # 11 - 10 -  9 -  8
    # |    |    |    | 
    #  7    6    5    4
    # |    |    |    |
    #  3    2    1    0
    # '''
    # xis = np.array([0.3,0.1,-0.1,-0.3])      # mod this
    # yis = np.array([0.3,0.1,-0.1,-0.3]*4)      # mod this
    # xis = np.repeat(xis, 4)           # mod this
    # diag[0,0] = np.sum(yis**2)
    # diag[1,1] = np.sum(xis**2)
    # diag[2,2] = np.sum(xis**2+yis**2)
    # Is = len(xis)*I
    # Is += m*diag
    # mass = m*16
    




In [14]:
f = open("generated.urdf", "w")
robot_name = 'tetris'          # mod this
Ixx, Iyy, Izz = Is[0,0],Is[1,1],Is[2,2]
collision_x, collision_y, collision_z = 0.6, 0.6, 0.08          # mod this
xs = xis
ys = yis

f.write('''<?xml version="1.0"?>
<robot name="{}">
    <properties arm="0.0397" kf="3.16e-10" km="7.94e-12" thrust2weight="2.25" max_speed_kmh="30" gnd_eff_coeff="11.36859" prop_radius="2.31348e-2" drag_coeff_xy="9.1785e-7" drag_coeff_z="10.311e-7" dw_coeff_1="2267.18" dw_coeff_2=".16" dw_coeff_3="-.11" />
    <link name="robot_base">
        <inertial>
            <origin rpy="0 0 0" xyz="0 0 0" />
            <inertia ixx="{}" ixz="0.0" ixy="0.0" iyy="{}" iyz="0.0" izz="{}" />
            <mass value="{}" />
        </inertial>
        '''.format(robot_name, Ixx, Iyy, Izz, mass))
f.write('''        
        <collision name="MultiRotor">
            <origin xyz="0.0 0.0 0.0" rpy="0.0 0.0 0.0"/>
            <geometry>
                <box size="{} {} {}" />
            </geometry>
        </collision>
        '''.format(collision_x, collision_y, collision_z))

# create modules
i = 1
for r in range(len(xs)):
    x = xs[r]
    y = ys[r]
    f.write('''
    <visual name="Shape{}">
        <origin xyz="{} {} 0.040000" rpy="0 -1.570796 0.000000" />
        <geometry>
            <mesh filename="./module_vis.dae" />
        </geometry>
        <material name="Shape0_material">
            <color rgba="0.1804 0.1804 0.1804 1.0" />
        </material>
    </visual>
            '''.format(i,x,y))
    i += 1
f.write('''
    </link>
        ''')

# create propellers
i = 1
offset_x = [0.05, -0.05, -0.05, 0.05]
offset_y = [-0.05, -0.05, 0.05, 0.05]
for r in range(len(xs)):
    for o in range(len(offset_x)):
        x = xs[r] + offset_x[o]
        y = ys[r] + offset_y[o]
        f.write('''
    <joint name="propeller{}" type="fixed">
        <axis xyz="0 0 1" />
        <parent link="robot_base" />
        <child link="propellerrespondable{}" />
        <origin xyz="{} {} 0.03" rpy="0 0 0" />
    </joint>
    <link name="propellerrespondable{}">
        <inertial>
            <origin rpy="0 0 0" xyz="-0.000000 0.000000 0.012000" />
            <inertia izz="1.5977926901062e-07" ixx="8.2325171633286e-08" ixy="-5.0583716089947e-21" iyz="-5.6129890954029e-16" iyy="8.2325156656747e-08" ixz="5.7933230213793e-16" />
            <mass value="0.0" />
        </inertial>
        <collision name="propeller_respondable{}">
            <origin xyz="-0.000000 0.000000 0.012000" rpy="3.141593 3.141593 1.865350" />
            <geometry>
                <cylinder length="0.0075362827628851" radius="0.024999996647239" />
            </geometry>
        </collision>
        <visual name="propeller_body{}">
            <origin xyz="-0.000000 0.000000 0.012000" rpy="0 0 0" />
            <geometry>
                <mesh filename="./propeller_vis.dae" />
            </geometry>
            <material name="propellerbody_material">
                <color rgba="1 0.5294 0 0.9" />
            </material>
        </visual>
    </link>
                '''.format(i,i,x,y,i,i,i))
        i += 1
f.write('''
</robot>
        ''')

f.close()